## One Million News Headlines

We are interested in the news headlines topics in the past two decades. What has been on the news?

We use the kaggle dataset that contains news headlines published over a period of seventeen years (2003-2019) by ABC (Australian Broadcasting Corp). 

We also use [this kaggle kernel](https://www.kaggle.com/ktattan/lda-and-document-similarity/notebook) and [this notebook on github](https://github.com/franciscadias/projects/blob/master/ABC_News.ipynb) as reference for data cleaning procedures and looking at top 10 words for the top 10 topics. 

In [1]:
import pandas as pd
import numpy as np

We first load the dataset and see what we have. This csv file is about 58MB in size, so we could consider storing is in a SQL database. Here we load it directly with pandas. 

In [2]:
df = pd.read_csv('abcnews-date-text.csv')
df.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


We then add columns for the year the headlines were published. This will be used in an analysis later.

In [3]:
df["year"] = df["publish_date"].astype(str).str[:4].astype(np.int64)

All rows with missing data is removed. The dataframe is then shuffled and re-indexed.

In [4]:
df.dropna(axis=0, inplace=True, subset=['headline_text'])

In [5]:
df = df.sample(frac=1.0)
df.reset_index(drop=True,inplace=True)
df.head()

,publish_date,headline_text,year
0,20110601,economic data weighs heavily on market,2011
1,20170821,adelaide cbd brothels complaints from residents,2017
2,20120710,sa government releases marine conservation areas,2012
3,20130801,vietnam bans social media from posting news on...,2013
4,20100917,police hunt sex assault suspect,2010


Import required NLP and text cleaning packages.

In [6]:
from time import time
import re
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chung\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chung\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\chung\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


We then clean the headline_text to clear it of websites, email addresses, and punctuation. The text is also made lowercase. After that, we tokenize the text by vectorizing the text corpus. 

A lemmatizer (as opposed to a stemmer) is used here so that we obtain real English words here. The parts of speech (POS) is found by the get_word_pos function. Alternatively, the spaCy lemmatizer can be used.

In [7]:
def clean(text):
    text = re.sub("((\S+)?(http(s)?)(\S+))|((\S+)?(www)(\S+))|((\S+)?(\@)(\S+)?)", " ", text)
    text = re.sub("[^a-zA-Z ]", "", text)
    text = text.lower()
    text = nltk.word_tokenize(text)
    return text

#https://www.machinelearningplus.com/nlp/lemmatization-examples-python/#wordnetlemmatizerwithappropriatepostag
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def lemma(text):
    return [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in text]

def remove_stop_words(text):
    return [word for word in text if word not in stop_words]

def apply_all(text):
    return lemma(remove_stop_words(clean(text)))

For such a large dataset, the difference in time for stemming and lemmatizing words is quite large. Stemming takes about 4.8min, whereas lemmatizing takes 47min, a 10x difference. Using the first 10000 lines of the dataset takes about 0.45min. 

In [8]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
tokenized = []
t1 = time()
for line in df['headline_text'].head(100000):
    txt = nlp(line)
    b = [t.lemma_ for t in txt if t.lemma_ not in stop_words]
    tokenized.append(b)
t2 = time()
print("Time to clean and tokenize %.2e articles: %.2f min." 
      %( len(tokenized), (t2-t1)/60))

print(tokenized[:5])

Time to clean and tokenize 1.00e+05 articles: 3.66 min.
[['economic', 'data', 'weigh', 'heavily', 'market'], ['adelaide', 'cbd', 'brothel', 'complaint', 'resident'], ['sa', 'government', 'release', 'marine', 'conservation', 'area'], ['vietnam', 'ban', 'social', 'medium', 'post', 'news', 'online'], ['police', 'hunt', 'sex', 'assault', 'suspect']]


In [9]:
# t1 = time()
# a= df['headline_text'].head(100000).apply(apply_all)
# t2 = time()
# print("Time to clean and tokenize %.2e articles: %.2f min." %( len(a), (t2-t1)/60))

# a.head(10)

Get word frequency

In [10]:
from nltk import FreqDist

# first get a list of all words
all_words = [word for item in list(tokenized) for word in item]

# use nltk fdist to get a frequency distribution of all words
fdist = FreqDist(all_words)
len(fdist) # number of unique words

32406

In [11]:
dfa = pd.DataFrame()
dfa['text'] = tokenized
dfa.head()

,text
0,"[economic, data, weigh, heavily, market]"
1,"[adelaide, cbd, brothel, complaint, resident]"
2,"[sa, government, release, marine, conservation..."
3,"[vietnam, ban, social, medium, post, news, onl..."
4,"[police, hunt, sex, assault, suspect]"


We use Latent Dirichlet Allocation (LDA), an unsupervised generative model that assigns topic distributions to documents. 

`LDA does not work well with very short documents, like twitter feeds, as explained here [1] [2], which is why we dropped articles under 40 tokens previously. Very briefly, this is because the model infers parameters from observations and if there are not enough observations (words) in a document, the model performs poorly.'

`Unlike the word2vec algorithm, which performs extremely well with full structured sentences, LDA is a bag of words model, meaning word order in a document doesnt count. This also means that stopwords and rare words should be excluded, so that the model doesnt overcompensate for very frequent words and very rare words, both of which do not contribute to general topics.'

In [12]:
import gensim

dictionary = gensim.corpora.Dictionary(dfa['text'])
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in dfa['text']]

In [13]:
t1 = time()
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=10, 
                                       id2word=dictionary,                                    
                                       passes=10,
                                       workers=3)
t2 = time()
print('LDA model trained in %.1f min.' % ((t2-t1)/60))

LDA model trained in 2.8 min.


We save this model.

In [14]:
from gensim.test.utils import datapath
lda_model.save('one_mil_model1')

We print the top 10 words of each of the top 10 topics in descending order of importance.

In [15]:
for a, b in lda_model.print_topics(num_topics=-1, num_words=10):
    print('Topic ',a)
    print('---------')
    for word in b.split(' + '):
        num, key = word[:5], word[6:]
        print(key)
    print('\n')

Topic  0
---------
"new"
"work"
"move"
"fear"
"pay"
"concern"
"strike"
"force"
"start"
"worker"


Topic  1
---------
"find"
"year"
"school"
"ban"
"flood"
"three"
"road"
"miss"
"body"
"dog"


Topic  2
---------
"fire"
"south"
"north"
"public"
"close"
"hit"
"west"
"warn"
"abc"
"rain"


Topic  3
---------
"market"
"rise"
"price"
"'"
"rate"
"high"
"industry"
"job"
"share"
"new"


Topic  4
---------
"claim"
"us"
"say"
"police"
"pm"
"leader"
"iraq"
"australian"
"protest"
"law"


Topic  5
---------
"council"
"plan"
"govt"
"water"
"health"
"coast"
"urge"
"fund"
"government"
"labor"


Topic  6
---------
"-PRON-"
"win"
"back"
"world"
"open"
"cup"
"first"
"final"
"say"
"lead"


Topic  7
---------
"day"
"test"
"national"
"rural"
"australia"
"show"
"search"
"good"
"award"
"darwin"


Topic  8
---------
"interview"
"take"
"centre"
"country"
"need"
"change"
"community"
"top"
"hour"
"trump"


Topic  9
---------
"man"
"police"
"charge"
"court"
"kill"
"woman"
"murder"
"attack"
"crash"
"death"




A Scikit-learn implementation using the sklearn CountVectorizer to clean and tokenize the corpus. The same dataset extract (first 100,000 lines) is used here.

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

all_words = df.headline_text.head(100000)

In [17]:
vect = CountVectorizer(stop_words="english")
vect.fit(all_words)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [18]:
print(len(vect.vocabulary_))

37429


In [19]:
bag_of_words = vect.transform(all_words)

Here we use the online learning method, which differs from the batch learning method. 

In [20]:
lda = LatentDirichletAllocation(n_components=10, learning_method="online",
                                max_iter=10, random_state=0, n_jobs=3, 
                                verbose=2)
# be build the model and transform the data in one step
# computing transform takes some time,
# and we can save time by doing both at once.
document_topics = lda.fit_transform(bag_of_words)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    2.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   

[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concu

[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concu

[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concu

...

[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concu

[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concu

[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concu

iteration: 10 of max_iter: 10


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    9.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    4.6s finished


In [21]:
# for each topic (a row in the components_), sort the features (ascending).
# Invert rows with [:, ::-1] to make sorting descending
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
# get the feature names from the vectorizer:
feature_names = np.array(vect.get_feature_names())

Print out the 10 topics.

In [22]:
import mglearn
mglearn.tools.print_topics(topics=range(10), feature_names=feature_names,
                           sorting=sorting, topics_per_chunk=5, n_words=10)

C:\test\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
C:\test\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
police        sydney        australian    says          new           
australia     woman         report        death         charged       
water         killed        sa            wa            missing       
crash         south         home          plan          power         
car           hit           rural         farmers       labor         
melbourne     dies          boost         drug          rise          
day           adelaide      high          attack        plans         
charges       work          open          group         canberra      
sex           china         public        case          record        
guilty        workers       opposition    fears         search        


topic 5       topic 6       topic 7       topic 8       topic 9       
--------      --------      --------      --------      --------      
cour